In [1]:
import os

import anthropic
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build


SCOPES = ['https://www.googleapis.com/auth/calendar']



if os.path.exists('./auth/token.json'):
    creds = Credentials.from_authorized_user_file('./auth/token.json', SCOPES)
else:
    flow = InstalledAppFlow.from_client_secrets_file(
        './auth/credentials.json', SCOPES
    )
    creds = flow.run_local_server(port=0)
    with open('./auth/token.json', 'w') as token:
        token.write(creds.to_json())

service = build('calendar', 'v3', credentials=creds)

### Function Call (Tool Use) 복습

In [ ]:
client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=[
        {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            },
        }
    ],
    messages=[{"role": "user", "content": "What's the weather like in San Francisco?"}],
)

In [ ]:
response.content

### Function Call + Google Calendar API

In [ ]:
event = {
  'summary': '패스트캠퍼스 LLM 강의',
  'location': '서울 강남구 테헤란로 231 West동 6층, 7층',
  'start': {
    'dateTime': '2024-08-14T09:00:00+09:00',
    'timeZone': 'Asia/Seoul',
  },
  'end': {
    'dateTime': '2024-08-14T18:00:00+09:00',
    'timeZone': 'Asia/Seoul',
  }
}

In [ ]:
tools = [
    {
        "name": "create_event",
        "description": "Create new Google Calender Event",
        "input_schema": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": "Name of Google Calender Event"
                },
                "start": {
                    "type": "string",
                    "description": "Starting date of Google Calender Event in UTC+9 Time i.e. 2024-08-08T09:00:00+09:00"
                },
                "end": {
                    "type": "string",
                    "description": "Starting date of Google Calender Event in UTC+9 i.e. 2024-08-08T09:00:00+09:00"
                }
            },
            "required": ["summary"]
        }
    }
]

In [ ]:
def create_event(summary, start, end):
    event = {
        'summary': summary,
        'start': {
            'dateTime': start,
            'timeZone': 'Asia/Seoul',
        },
        'end': {
            'dateTime': end,
            'timeZone': 'Asia/Seoul',
        }
    }
    event = service.events().insert(calendarId='primary', body=event).execute()
    print('Event created: %s' % (event.get('htmlLink')))
    return event

In [ ]:
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=[
        {
            "role": "user",
            "content": "Create a calender event called Lunch starting from 2024.08.14 1pm"
        }
    ],
)

In [ ]:
response.content

In [ ]:
response.content[1].input

In [ ]:
summary = response.content[1].input['summary']
start = response.content[1].input['start']
end = response.content[1].input['end']
create_event(summary, start, end)

### End-to-End Function Call Workflow
1. Function 준비 (또는 작성)
2. Tools 정의
3. LLM API 호출
4. (필요 시) Function 실행 - 일반 발화가 필요한경우 실행 필요 X
5. LLM API 최종 호출

In [ ]:
PROMPT = '2024년 8월 14일 12시부터 1시간 짜리 점심 일정 생성해주세요.'

# 1. 함수 생성
def create_event(summary, start, end):
    event = {
        'summary': summary,
        'start': {
            'dateTime': start,
            'timeZone': 'Asia/Seoul',
        },
        'end': {
            'dateTime': end,
            'timeZone': 'Asia/Seoul',
        }
    }
    event = service.events().insert(calendarId='primary', body=event).execute()
    print('Event created: %s' % (event.get('htmlLink')))
    return event

# 2. tools 정의
tools = [
    {
        "name": "create_event",
        "description": "Create new Google Calender Event",
        "input_schema": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": "Name of Google Calender Event"
                },
                "start": {
                    "type": "string",
                    "description": "Starting date of Google Calender Event in UTC+9 Time i.e. 2024-08-08T09:00:00+09:00"
                },
                "end": {
                    "type": "string",
                    "description": "Starting date of Google Calender Event in UTC+9 Time i.e. 2024-08-08T09:00:00+09:00"
                }
            },
            "required": ["summary"]
        }
    }
]

# 3. LLM API 호출
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=[
        {
            "role": "user",
            "content": PROMPT
        }
    ],
)

# 4. (필요 시) Function Call 실행
summary = response.content[1].input['summary']
start = response.content[1].input['start']
end = response.content[1].input['end']
event = create_event(summary, start, end)

# 5. LLM API 최종 호출
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=[
        {
            "role": "user",
            "content": PROMPT
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": PROMPT
                },
                {
                    "type": "tool_use",
                    "id": response.content[1].id,
                    "name": "create_event",
                    "input": {"summary": summary, "start": start, "end": end}
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": response.content[1].id,
                    "content": f"Event created: {event.get('htmlLink')}"
                }
            ]
        }
    ],
)

In [ ]:
print(response.content[0].text)